In [83]:
# Exploratory Data Analysis (EDA)

import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import numpy as np


root = Path.cwd().parent
print(root)

c:\Users\sebas\PycharmProjects\Git\Seb_branch\institutional-roi-analysis


In [99]:
df.isna().sum()

code                                                 0
title                                                0
unit_id                                              0
distance                                             0
school.type                                          0
school.main_campus                                   0
credential.level                                     0
earnings.4_yr.overall_median_earnings                0
earnings.4_yr.working_not_enrolled.overall_count     0
debt.staff_grad_plus.all.eval_inst.median            0
debt.staff_grad_plus.all.eval_inst.median_payment    0
school.name                                          0
dtype: int64

In [81]:
df = pd.read_csv(root/"data"/"raw"/"scorecard"/"test.csv").drop(columns="Unnamed: 0")

In [44]:
df.describe()

,Unnamed: 0,code,unit_id,distance,school.main_campus,credential.level,earnings.4_yr.overall_median_earnings,earnings.4_yr.working_not_enrolled.overall_count,debt.staff_grad_plus.all.eval_inst.median
count,346.00000,346.000000,346.000000,346.000000,346.000000,346.000000,346.000000,346.000000,346.000000
mean,172.50000,3799.416185,462743.008671,1.398844,0.361272,2.208092,42463.583815,283.670520,21459.578035
std,100.02583,1761.477353,22433.957322,0.807500,0.481065,1.471485,22315.934315,654.265594,18148.184832
min,0.00000,301.000000,417327.000000,0.000000,0.000000,1.000000,12625.000000,16.000000,3167.000000
25%,86.25000,1446.000000,445345.000000,1.000000,0.000000,1.000000,25291.000000,52.000000,9500.000000
50%,172.50000,5106.000000,457053.000000,1.000000,0.000000,2.000000,36162.500000,116.000000,17104.000000
75%,258.75000,5135.000000,487311.000000,2.000000,1.000000,3.000000,54349.500000,237.000000,25900.000000
max,345.00000,5401.000000,497222.000000,3.000000,1.000000,8.000000,148622.000000,6804.000000,144259.000000


In [50]:
earning_count = go.Figure(
    go.Histogram(
        x=df["earnings.4_yr.working_not_enrolled.overall_count"],
        xbins=dict(start=0, end=6804, size=136.08)
    )
)

earning_count.show()


In [47]:
px.histogram(df, x="earnings.4_yr.overall_median_earnings")

due to skewness, i will try to np.log on median income

In [102]:
px.histogram(df, x=np.log1p(df["earnings.4_yr.overall_median_earnings"]))

log is successful so i will add it as a new column

In [86]:
px.histogram(
    df,
    x="debt.staff_grad_plus.all.eval_inst.median_payment"
)

i will try to correct skew with log for median debt payments, and median total debt

In [100]:
px.histogram(df, x=np.log1p(df["debt.staff_grad_plus.all.eval_inst.median_payment"]))

In [90]:
dfx = df.loc[df["earnings.4_yr.working_not_enrolled.overall_count"] >= 30]

In [91]:
lt50=len(df) - len(dfx)
lt50


33

applying a 30 sudent threshold removes 33 instances (≈10%)

In [ ]:
dfx["log_median_earnings"]=np.log1p(dfx["earnings.4_yr.overall_median_earnings"])
dfx["log_median_debt_payment"]=np.log1p(dfx["debt.staff_grad_plus.all.eval_inst.median_payment"]*12)
dfx["log_median_debt"]=np.log1p(dfx["debt.staff_grad_plus.all.eval_inst.median"])
# dfx

In [139]:
dfx["debt_burden"] = (dfx["debt.staff_grad_plus.all.eval_inst.median_payment"]*12)/dfx["earnings.4_yr.overall_median_earnings"]
dfx.loc[dfx["debt_burden"]>=0.2]

,code,title,unit_id,distance,school.type,school.main_campus,credential.level,earnings.4_yr.overall_median_earnings,earnings.4_yr.working_not_enrolled.overall_count,debt.staff_grad_plus.all.eval_inst.median,debt.staff_grad_plus.all.eval_inst.median_payment,school.name,log_median_earnings,log_median_debt_payment,log_median_debt,debt_burden,log_debt_burden
74,2201,Law.,442295,1,"Private, nonprofit",1,7,69213.0,116.0,144259.0,1641,Ave Maria School of Law,11.144944,9.887968,11.879366,0.284513,0.250380
115,1304,Educational Administration and Supervision.,449038,3,"Private, for-profit",0,5,45764.0,209.0,81000.0,921,Strayer University-Florida,10.731253,9.310367,11.302204,0.241500,0.216320
117,4404,Public Administration.,449038,3,"Private, for-profit",0,5,55281.0,173.0,81723.0,930,Strayer University-Florida,10.920185,9.320091,11.311091,0.201878,0.183885
118,5107,Health and Medical Administrative Services.,449038,3,"Private, for-profit",0,5,48777.0,224.0,81207.0,924,Strayer University-Florida,10.795014,9.313619,11.304757,0.227320,0.204833
124,5210,Human Resources Management and Services.,449038,3,"Private, for-profit",0,5,55030.0,324.0,80998.0,921,Strayer University-Florida,10.915634,9.310367,11.302180,0.200836,0.183018


In [140]:
dfx.loc[dfx["debt_burden"]<=0.02]

,code,title,unit_id,distance,school.type,school.main_campus,credential.level,earnings.4_yr.overall_median_earnings,earnings.4_yr.working_not_enrolled.overall_count,debt.staff_grad_plus.all.eval_inst.median,debt.staff_grad_plus.all.eval_inst.median_payment,school.name,log_median_earnings,log_median_debt_payment,log_median_debt,debt_burden,log_debt_burden
191,5109,"Allied Health Diagnostic, Intervention, and Tr...",475714,1,"Private, for-profit",1,1,64877.0,82.0,9500.0,101,American Medical Academy,11.080248,7.100027,9.159047,0.018682,0.018509
233,1204,Cosmetology and Related Personal Grooming Serv...,483896,1,"Private, for-profit",1,1,21151.0,65.0,3167.0,34,Parisian Spa Institute,9.959442,6.011267,8.060540,0.019290,0.019106
236,1101,"Computer and Information Sciences, General.",484473,3,Public,0,3,108528.0,111.0,15000.0,159,University of Florida-Online,11.594763,7.553811,9.615805,0.017581,0.017428


In [130]:
px.histogram(
    dfx,
    x="debt_burden"
)

In [135]:
dfx["log_debt_burden"] = np.log1p(dfx["debt_burden"])

In [137]:
dfx

,code,title,unit_id,distance,school.type,school.main_campus,credential.level,earnings.4_yr.overall_median_earnings,earnings.4_yr.working_not_enrolled.overall_count,debt.staff_grad_plus.all.eval_inst.median,debt.staff_grad_plus.all.eval_inst.median_payment,school.name,log_median_earnings,log_median_debt_payment,log_median_debt,debt_burden,log_debt_burden
0,5108,Allied Health and Medical Assisting Services.,417327,1,"Private, nonprofit",1,2,34153.0,30.0,25029.0,265,City College-Altamonte Springs,10.438606,8.064636,10.127790,0.093110,0.089027
2,1402,"Aerospace, Aeronautical and Astronautical Engi...",426314,3,"Private, nonprofit",0,3,86529.0,202.0,26312.0,279,Embry-Riddle Aeronautical University-Worldwide,11.368235,8.116118,10.177780,0.038692,0.037962
4,4303,Homeland Security.,426314,3,"Private, nonprofit",0,3,64823.0,45.0,20500.0,217,Embry-Riddle Aeronautical University-Worldwide,11.079416,7.864804,9.928180,0.040171,0.039385
5,4706,Vehicle Maintenance and Repair Technologies.,426314,3,"Private, nonprofit",0,2,76825.0,60.0,15375.0,163,Embry-Riddle Aeronautical University-Worldwide,11.249285,7.578657,9.640498,0.025460,0.025142
6,4706,Vehicle Maintenance and Repair Technologies.,426314,3,"Private, nonprofit",0,3,85596.0,91.0,22662.0,240,Embry-Riddle Aeronautical University-Worldwide,11.357394,7.965546,10.028445,0.033646,0.033093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,5202,"Business Administration, Management and Operat...",496973,2,"Private, nonprofit",0,2,37295.0,71.0,29440.0,312,Herzing University-Tampa,10.526615,8.227910,10.290110,0.100389,0.095664
342,5202,"Business Administration, Management and Operat...",496973,2,"Private, nonprofit",0,3,55767.0,158.0,33791.0,358,Herzing University-Tampa,10.928938,8.365440,10.427950,0.077035,0.074212
343,5202,"Business Administration, Management and Operat...",496973,2,"Private, nonprofit",0,5,49866.0,173.0,41000.0,466,Herzing University-Tampa,10.817095,8.629092,10.621327,0.112141,0.106287
344,5138,"Registered Nursing, Nursing Administration, Nu...",496991,1,"Private, for-profit",0,2,67737.0,958.0,24808.0,263,Galen Health Institutes-Miami Campus,11.123388,8.057061,10.118921,0.046592,0.045539
